In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from src.gem.yeast8model import Yeast8Model
from src.calc.ablation import get_ablation_ratio
from src.calc.flux import compare_fluxes
from src.calc.growth import get_exch_saturation
from src.viz.bar import ablation_barplot, compare_ablation_times

# Construct models of cells of interest, optimise

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.solution = wt.optimize()

BY4741 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
BY4741.make_auxotroph("BY4741")
BY4741.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4741.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4741.solution = BY4741.optimize()

BY4742 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
BY4742.make_auxotroph("BY4742")
BY4742.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
BY4742.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
BY4742.solution = BY4742.optimize()

zwf1 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
zwf1.make_auxotroph("BY4741")
zwf1.knock_out_list(["YNL241C"])
zwf1.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
zwf1.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
zwf1.solution = zwf1.optimize()

tsa2 = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
tsa2.make_auxotroph("BY4742")
tsa2.knock_out_list(["YDR453C"])
tsa2.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
tsa2.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
tsa2.solution = tsa2.optimize()

# Ablate

TODO: For loop all this?  It's repetitive.

## Wild type

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

## BY4741

In [ ]:
BY4741.ablation_result = BY4741.ablate()

fig, ax = plt.subplots()
ablation_barplot(BY4741.ablation_result, ax)
ax.set_title('BY4741 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(BY4741.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4741.ablation_result, ax_fc)

## BY4742

In [ ]:
BY4742.ablation_result = BY4742.ablate()

fig, ax = plt.subplots()
ablation_barplot(BY4742.ablation_result, ax)
ax.set_title('BY4742 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(BY4742.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt.ablation_result, BY4742.ablation_result, ax_fc)

In [ ]:
zwf1.ablation_result = zwf1.ablate()
fig, ax = plt.subplots()
ablation_barplot(zwf1.ablation_result, ax)
ax.set_title('zwf1$\Delta$ BY4741 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(zwf1.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4741.ablation_result, zwf1.ablation_result, ax_fc)

In [ ]:
tsa2.ablation_result = tsa2.ablate()
fig, ax = plt.subplots()
ablation_barplot(tsa2.ablation_result, ax)
ax.set_title('tsa2$\Delta$ BY4742 (in supplemented media)')
plt.show()

ratio = get_ablation_ratio(tsa2.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(BY4742.ablation_result, tsa2.ablation_result, ax_fc)

In [ ]:
wt.ablation_result

In [ ]:
BY4741.ablation_result

In [ ]:
BY4742.ablation_result

In [ ]:
zwf1.ablation_result

In [ ]:
tsa2.ablation_result

In [ ]:
wt_ablation_result_orig = wt.ablation_result.copy()

## Add amino acids to minimal media

In [ ]:
amino_exch_list = [
    'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1883',
    'r_1889', 'r_1891', 'r_1810', 'r_1893', 'r_1897',
    'r_1899', 'r_1900', 'r_1902', 'r_1903', 'r_1904',
    'r_1906', 'r_1911', 'r_1912', 'r_1913', 'r_1914',
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(amino_exch_list)
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
for reaction_id in wt.model.medium.keys():
    print(wt.model.reactions.get_by_id(reaction_id).name)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all amino acids supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

> You can see that it takes less time to make proteins.

## Add nucleotides to minimal media

In [ ]:
nucl_exch_list = [
    'r_1639', 'r_1705', 'r_1818', 'r_2090'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(nucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all NTPs supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Add deoxyribonucleotides to media

In [ ]:
deoxnucl_exch_list = [
    'r_1643', 'r_1702', 'r_1820', 'r_2073'
]

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.add_media_components(deoxnucl_exch_list)

In [ ]:
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with all dNTPs supplemented')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

## Carbon sources

Pyruvate

In [ ]:
wt.reset_to_file()
wt.unrestrict_growth()
wt.add_media_components(['r_2033'])
# Set uptake rate (under saturation) to get a growth rate consistent with experiments.
wt.model.reactions.get_by_id('r_2033').bounds = (-5.476, 0)
wt.remove_media_components(['r_1714', 'r_1714_REV'])
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with pyruvate as carbon source')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
wt.ablation_result

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

Glucose limitation

In [ ]:
glc_exch_rate = 1.329
# Simulates 10 mg/L by assuming a 3-hour doubling time
# and using the saturation curve at the end of this notebook

wt.reset_to_file()
wt.unrestrict_growth()
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.ablation_result = wt.ablate()

fig, ax = plt.subplots()
ablation_barplot(wt.ablation_result, ax)
ax.set_title('Wild type, with glucose limitation')
plt.show()

ratio = get_ablation_ratio(wt.ablation_result)
print(ratio)

In [ ]:
fig_fc, ax_fc = plt.subplots()
compare_ablation_times(wt_ablation_result_orig, wt.ablation_result, ax_fc)

# Compare fluxes

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, zwf1)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(BY4741, tsa2)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
for rxn_id, flux in diff_fluxes_sorted.items():
    print(f'{rxn_id}, {wt.model.reactions.get_by_id(rxn_id).name}, {flux}')

In [ ]:
for reaction_id in BY4741.model.medium.keys():
    print(BY4741.model.reactions.get_by_id(reaction_id).name)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, BY4741)

In [ ]:
diff_fluxes_sorted = compare_fluxes(wt, wt)

In [ ]:
diff_fluxes_sorted

## Respiration and fermentation

In [ ]:
models = {
    "wild type": wt,
    "BY4741": BY4741,
    "zwf1": zwf1,
    "tsa2": tsa2,
}
biomass_components = ["carbohydrate", "protein"]
exchanges = {
    "O2": "r_1992_REV",
    "CO2": "r_1672",
    "ethanol": "r_1761",
}

for strain, model in models.items():
    print(strain)
    for biomass_component in biomass_components:
        print(biomass_component)
        for exchange_species, exchange_id in exchanges.items():
            flux = model.ablation_reaction_fluxes[biomass_component][exchange_id]
            print(f"{exchange_species}: {flux}")
        print("\n")
    print("----------")